In [1]:
import pandas as pd
import numpy as np
import copy

In [2]:
# genes-pathways annotation

path = './KEGG_pathways/20230205_kegg_hsa.gmt'

files = open(path,encoding='utf-8')

files = files.readlines()

paways_genes_dict = {}
for i in files: 
    paways_genes_dict[i.split('\t')[0].split('_')[0]] = i.replace('\n','').split('\t')[2:] 


In [3]:
#mirna-pathways annotation
path = './KEGG_pathways/kegg_anano.txt'

files = open(path,encoding='utf-8')

files = files.readlines()

paways_mirna_dict = {}
for i in files:
     keys = i.split(',')[0].split('|')[1]
     values1 = i.split(',')[1:-1]
     values2 =  i.split(',')[-1].replace('\n','')
     values1.append(values2)
     values1 =list(set(values1)) 
     paways_mirna_dict[keys] = values1

In [4]:
union_kegg = list(set(paways_genes_dict.keys()).intersection(set(paways_mirna_dict.keys())))

In [5]:
paways_genes_dicts ={}
paways_mirna_dicts ={}

for i in union_kegg:
    paways_genes_dicts[i] = paways_genes_dict[i]
    
for i in union_kegg:
    paways_mirna_dicts[i] = paways_mirna_dict[i]    

In [6]:

genes_existed_pathway = []

mirna_existed_pathway = []

for index in paways_genes_dicts.keys():
    genes_existed_pathway = genes_existed_pathway+ list(paways_genes_dicts[index])
genes_existed_pathway = set(genes_existed_pathway)


for index in paways_mirna_dicts.keys():
    mirna_existed_pathway = mirna_existed_pathway+ list(paways_mirna_dicts[index])
mirna_existed_pathway = set(mirna_existed_pathway)



In [7]:
print(len(genes_existed_pathway))
print(len(mirna_existed_pathway))

7768
812


In [8]:
#Loading data

miRNA_data = pd.read_csv("./AML_data/miRNA_data.csv",index_col = 0)

mRNA_data = pd.read_csv("./AML_data/mRNA_data.csv",index_col = 0)

example_case = pd.read_csv('./AML_data/response.csv',index_col=0)

In [9]:
print(mRNA_data.shape)
print(miRNA_data.shape)

(221, 2000)
(221, 100)


In [12]:
# mRNA_data

In [10]:
# example_case

In [13]:
union_gene_miRNA = list(miRNA_data.columns)
union_gene_mRNA = list(mRNA_data.columns)

In [14]:
pathway_union = list(paways_genes_dicts.keys())
len(pathway_union)

238

In [15]:
mask_list = [union_gene_mRNA]

gene_pathway_bp_dfs = []


for i in range(len(mask_list)):
    pathways_genes = np.zeros((len(pathway_union), len(mask_list[i]))) 
    for p  in pathway_union:
        gs = paways_genes_dicts[p]
        g_inds = [mask_list[i].index(g) for g in gs if g in mask_list[i]]
        p_ind = pathway_union.index(p)
        pathways_genes[p_ind, g_inds] = 1
    gene_pathway_bp = pd.DataFrame(pathways_genes, index=pathway_union, columns=mask_list[i])
    
    gene_pathway_bp_dfs.append(gene_pathway_bp)
    

pathways_genes = np.zeros((len(pathway_union), len(union_gene_miRNA))) 
for p  in pathway_union:
    gs = paways_mirna_dicts[p]
    g_inds = [union_gene_miRNA.index(g) for g in gs if g in union_gene_miRNA]
    p_ind = pathway_union.index(p)
    pathways_genes[p_ind, g_inds] = 1
gene_pathway_bp = pd.DataFrame(pathways_genes, index=pathway_union, columns=union_gene_miRNA)

gene_pathway_bp_dfs.append(gene_pathway_bp)
    


In [14]:
# len(gene_pathway_bp_dfs)

In [15]:
# gene_pathway_bp_dfs

In [16]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras.initializers import glorot_uniform, Initializer
from tensorflow.keras.layers import Input, Dense, Dropout, Embedding, GlobalAveragePooling1D,Layer
from tensorflow.keras import initializers,activations,regularizers
from tensorflow.keras.regularizers import Regularizer
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras import backend as K
from tensorflow.keras.regularizers import l2
from tensorflow.python.framework.ops import disable_eager_execution
 
tf.compat.v1.disable_eager_execution()

In [17]:
class Biological_module(Layer):
    def __init__(self, units, mapp=None, nonzero_ind=None, kernel_initializer='glorot_uniform', W_regularizer=None,
                 activation='tanh', use_bias=True,bias_initializer='zeros', bias_regularizer=None,
                 bias_constraint=None,**kwargs):
        
        self.units = units
        self.activation = activation
        self.mapp = mapp
        self.nonzero_ind = nonzero_ind
        self.use_bias = use_bias
        
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.kernel_regularizer = regularizers.get(W_regularizer)
        self.bias_initializer = initializers.get(bias_initializer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.activation_fn = activations.get(activation)
        super(Biological_module, self).__init__(**kwargs)

        
    def build(self, input_shape):
        
        input_dim = input_shape[1]
   

        if not self.mapp is None:
            self.mapp = self.mapp.astype(np.float32)

   
        if self.nonzero_ind is None:
            nonzero_ind = np.array(np.nonzero(self.mapp)).T
            self.nonzero_ind = nonzero_ind

        self.kernel_shape = (input_dim, self.units)
        

        nonzero_count = self.nonzero_ind.shape[0]   


        self.kernel_vector = self.add_weight(name='kernel_vector',
                                             shape=(nonzero_count,),
                                             initializer=self.kernel_initializer,
                                             regularizer=self.kernel_regularizer,
                                             trainable=True)
        if self.use_bias:
            self.bias = self.add_weight(shape=(self.units,),
                                        initializer=self.bias_initializer,
                                        name='bias',
                                        regularizer=self.bias_regularizer
                                        )
        else:
            self.bias = None

        super(Biological_module, self).build(input_shape)  
      

    def call(self, inputs):
        
        
        trans = tf.scatter_nd(tf.constant(self.nonzero_ind, tf.int32), self.kernel_vector,
                           tf.constant(list(self.kernel_shape)))
    
        output = K.dot(inputs, trans)
        
    
        if self.use_bias:
            output = K.bias_add(output, self.bias)
            
        if self.activation_fn is not None:
            output = self.activation_fn(output)

        return output

    def get_config(self):
        config = {
            'units': self.units,
            'activation': self.activation,
            'use_bias': self.use_bias,
            'nonzero_ind': np.array(self.nonzero_ind),
          
            'bias_initializer': initializers.serialize(self.bias_initializer),
            'bias_regularizer': regularizers.serialize(self.bias_regularizer),

            'kernel_initializer': initializers.serialize(self.kernel_initializer),
            'W_regularizer': regularizers.serialize(self.kernel_regularizer),

        }
        base_config = super(Biological_module, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def compute_output_shape(self, input_shape):
      
        return (input_shape[0], self.units)

In [18]:
class Self_Attention(Layer):
 
    def __init__(self, output_dim,  W_regularizer=None,**kwargs):
        self.output_dim = output_dim
        self.kernel_regularizer = regularizers.get(W_regularizer)
        super(Self_Attention, self).__init__(**kwargs)
 
    def build(self, input_shape):
        self.kernel = self.add_weight(name='kernel',
                                      shape=(3,input_shape[1], self.output_dim),
                                      initializer='uniform',
                                      regularizer=self.kernel_regularizer,
                                      trainable=True)
 
        super(Self_Attention, self).build(input_shape)  
 
    def call(self, x):
        WQ = K.dot(x, self.kernel[0])
        WK = K.dot(x, self.kernel[1])
        WV = K.dot(x, self.kernel[2])
 

        print("K.permute_dimensions(WK.shape",(K.permute_dimensions(WK,[1,0]).shape))
 
        QK =  K.dot(K.permute_dimensions(WK,[1,0]),WQ)
    
 
        QK = QK / (64**0.5)
 
        QK = K.softmax(QK)
 
        print("QK.shape",QK.shape)
 
        V = K.dot(WV,QK)
        
        print(V.shape)
 
        return V

    def get_config(self):
        config = {
          
            'bias_regularizer': regularizers.serialize(self.bias_regularizer),


        }
        base_config = super(Self_Attention, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
 
    def compute_output_shape(self, input_shape):
 
        return (input_shape[0],input_shape[1],self.output_dim)
 

In [20]:
def create_model(mRNA_data,miRNA_data):
    

    
    S_inputs_mRNA = Input(shape=(mRNA_data.shape[1],), dtype='float32',name= 'mRNA_inputs')
  
    S_inputs_miRNA = Input(shape=(miRNA_data.shape[1],), dtype='float32',name= 'miRNA_inputs')
    

   
    h0_mRNA = Biological_module(gene_pathway_bp_dfs[0].shape[0],mapp =gene_pathway_bp_dfs[0].values.T, name = 'h0_mRNA',W_regularizer=l2(0.001))(S_inputs_mRNA)

    
    h0_miRNA = Biological_module(gene_pathway_bp_dfs[1].shape[0],mapp =gene_pathway_bp_dfs[1].values.T, name = 'h0_miRNA',W_regularizer=l2(0.001))(S_inputs_miRNA)



    atten1 = Self_Attention(64,W_regularizer=l2(0.003))(h0_mRNA)
    atten2 = Self_Attention(64,W_regularizer=l2(0.003))(h0_miRNA)
    
    feature_tal = tf.keras.layers.concatenate([atten1,atten2])

    
    h4 = tf.keras.layers.Dense(32,activation='tanh')(feature_tal)
    
    h5 = tf.keras.layers.Dense(1,activation='sigmoid')(h4)
    

    model = Model(inputs=[S_inputs_mRNA,S_inputs_miRNA], outputs=h5)

    model.summary()

    opt = tf.keras.optimizers.Adam(learning_rate = 0.0001,decay=0.0001) 
    model.compile(optimizer=opt,
                  loss='binary_crossentropy',
                  metrics=['acc'])
    return model

In [21]:
#Evaluation function
import sklearn
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import average_precision_score
   

from sklearn.metrics import precision_recall_curve
def get_metrics(true_score,pre_score,pre_probe):
    
  
    fpr, tpr, thresholds = metrics.roc_curve(true_score, pre_probe, pos_label=1)
   
    auc = metrics.auc(fpr, tpr)
    
    aupr = average_precision_score(true_score, pre_probe)
    
    pre, rec, thresholds = precision_recall_curve(true_score, pre_probe)    
    auprc  = metrics.auc(rec, pre)
    
    
    accuracy = accuracy_score(true_score,pre_score)
    
    f1 = metrics.f1_score(true_score, pre_score)
    
    precision = metrics.precision_score(true_score,pre_score)
    
    recall = metrics.recall_score(true_score,pre_score)
    
     
    print( print(confusion_matrix(true_score,pre_score)))
    return precision,accuracy,recall,f1,auc,aupr,auprc


def evaluates(y_test, y_pred):
    
    auc = metrics.roc_auc_score(y_test,y_pred)
    
    aupr = average_precision_score(y_test, y_pred)
    
    precision, recall, thresholds = precision_recall_curve(y_test, y_pred)    
    auprc  = metrics.auc(recall, precision)
    
    pp = [1 if index>=0.5  else 0 for index in  y_pred ]
    
    pre = metrics.precision_score(y_test,pp)
    
    f1 = metrics.f1_score(y_test,pp)
    
    rec = metrics.recall_score(y_test,pp)
    
    acc = metrics.accuracy_score(y_test,pp)
    
    print(confusion_matrix(y_test,pp))
    return pre,acc,rec,f1,auc,aupr,auprc

In [22]:
y = example_case['response'].values
n_samples =example_case['response'].values

print(len(n_samples),n_samples.sum(),(len(n_samples) -n_samples.sum()))

x_0 =  len(n_samples) / (2*  (len(n_samples) -n_samples.sum()))
x_1 =  len(n_samples) / (2*  n_samples.sum())

print(x_0,x_1)

221 120 101
1.0940594059405941 0.9208333333333333


In [23]:
#Five-fold cross validation
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=1030) 

kfscore = []
p = 0
for train_index, test_index in skf.split(mRNA_data.values,y):


    mRNA_train_x = mRNA_data.values[train_index]
    mRNA_test_x  = mRNA_data.values[test_index]

    miRNA_train_x = miRNA_data.values[train_index]
    miRNA_test_x  = miRNA_data.values[test_index]

    train_y  = y[train_index]
    test_y   = y[test_index]

    model = create_model(mRNA_data,miRNA_data)
    model.fit( { "mRNA_inputs": mRNA_train_x, 'miRNA_inputs':miRNA_train_x},train_y,
                 epochs=130,batch_size = 64,class_weight = {0:x_0,1:x_1})  

    y_pred = model.predict({"mRNA_inputs": mRNA_test_x,'miRNA_inputs':miRNA_test_x})

    y_score = [1 if index>=0.5  else 0 for index in  y_pred]

    evaluate_epoch = get_metrics(test_y,y_score,y_pred)
    print(evaluate_epoch)

    kfscore.append(evaluate_epoch)
    
results = list(np.array(kfscore).sum(axis= 0)/5.0)
print('Cross validated results :  ACC = {}, REC = {}, F1 = {}, AUC = {}, AUPR ={}'.format(results[1],results[2],results[3],results[4],results[5]))

K.permute_dimensions(WK.shape (64, None)
QK.shape (64, 64)
(None, 64)
K.permute_dimensions(WK.shape (64, None)
QK.shape (64, 64)
(None, 64)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 mRNA_inputs (InputLayer)       [(None, 2000)]       0           []                               
                                                                                                  
 miRNA_inputs (InputLayer)      [(None, 100)]        0           []                               
                                                                                                  
 h0_mRNA (Biological_module)    (None, 238)          6087        ['mRNA_inputs[0][0]']            
                                                                                                  
 h0_miRNA (Biological_module)   (None, 238)          

176/176 [==============================] - 0s 527us/sample - loss: 0.5983 - acc: 0.7614
Epoch 55/130
176/176 [==============================] - 0s 272us/sample - loss: 0.5944 - acc: 0.7557
Epoch 56/130
176/176 [==============================] - 0s 476us/sample - loss: 0.5888 - acc: 0.7614
Epoch 57/130
176/176 [==============================] - 0s 448us/sample - loss: 0.5843 - acc: 0.7614
Epoch 58/130
176/176 [==============================] - 0s 368us/sample - loss: 0.5797 - acc: 0.7614
Epoch 59/130
176/176 [==============================] - 0s 272us/sample - loss: 0.5750 - acc: 0.7614
Epoch 60/130
176/176 [==============================] - 0s 266us/sample - loss: 0.5706 - acc: 0.7727
Epoch 61/130
176/176 [==============================] - 0s 255us/sample - loss: 0.5661 - acc: 0.7727
Epoch 62/130
176/176 [==============================] - 0s 261us/sample - loss: 0.5614 - acc: 0.7670
Epoch 63/130
176/176 [==============================] - 0s 261us/sample - loss: 0.5573 - acc: 0.7784
Epo

D:\Anaconda3\lib\site-packages\keras\engine\training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


K.permute_dimensions(WK.shape (64, None)
QK.shape (64, 64)
(None, 64)
K.permute_dimensions(WK.shape (64, None)
QK.shape (64, 64)
(None, 64)
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 mRNA_inputs (InputLayer)       [(None, 2000)]       0           []                               
                                                                                                  
 miRNA_inputs (InputLayer)      [(None, 100)]        0           []                               
                                                                                                  
 h0_mRNA (Biological_module)    (None, 238)          6087        ['mRNA_inputs[0][0]']            
                                                                                                  
 h0_miRNA (Biological_module)   (None, 238)        

177/177 [==============================] - 0s 231us/sample - loss: 0.5926 - acc: 0.7853
Epoch 55/130
177/177 [==============================] - 0s 231us/sample - loss: 0.5882 - acc: 0.7853
Epoch 56/130
177/177 [==============================] - 0s 220us/sample - loss: 0.5812 - acc: 0.7910
Epoch 57/130
177/177 [==============================] - 0s 248us/sample - loss: 0.5751 - acc: 0.7853
Epoch 58/130
177/177 [==============================] - 0s 231us/sample - loss: 0.5695 - acc: 0.7853
Epoch 59/130
177/177 [==============================] - 0s 242us/sample - loss: 0.5642 - acc: 0.7910
Epoch 60/130
177/177 [==============================] - 0s 225us/sample - loss: 0.5595 - acc: 0.7910
Epoch 61/130
177/177 [==============================] - 0s 225us/sample - loss: 0.5534 - acc: 0.7910
Epoch 62/130
177/177 [==============================] - 0s 225us/sample - loss: 0.5481 - acc: 0.7910
Epoch 63/130
177/177 [==============================] - 0s 242us/sample - loss: 0.5444 - acc: 0.7910
Epo

...

177/177 [==============================] - 0s 237us/sample - loss: 0.6136 - acc: 0.7627
Epoch 55/130
177/177 [==============================] - 0s 270us/sample - loss: 0.6081 - acc: 0.7627
Epoch 56/130
177/177 [==============================] - 0s 203us/sample - loss: 0.6031 - acc: 0.7571
Epoch 57/130
177/177 [==============================] - 0s 242us/sample - loss: 0.5961 - acc: 0.7684
Epoch 58/130
177/177 [==============================] - 0s 225us/sample - loss: 0.5907 - acc: 0.7684
Epoch 59/130
177/177 [==============================] - 0s 203us/sample - loss: 0.5852 - acc: 0.7684
Epoch 60/130
177/177 [==============================] - 0s 237us/sample - loss: 0.5798 - acc: 0.7684
Epoch 61/130
177/177 [==============================] - 0s 192us/sample - loss: 0.5739 - acc: 0.7684
Epoch 62/130
177/177 [==============================] - 0s 248us/sample - loss: 0.5693 - acc: 0.7571
Epoch 63/130
177/177 [==============================] - 0s 265us/sample - loss: 0.5641 - acc: 0.7797
Epo

D:\Anaconda3\lib\site-packages\keras\engine\training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


K.permute_dimensions(WK.shape (64, None)
QK.shape (64, 64)
(None, 64)
K.permute_dimensions(WK.shape (64, None)
QK.shape (64, 64)
(None, 64)
Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 mRNA_inputs (InputLayer)       [(None, 2000)]       0           []                               
                                                                                                  
 miRNA_inputs (InputLayer)      [(None, 100)]        0           []                               
                                                                                                  
 h0_mRNA (Biological_module)    (None, 238)          6087        ['mRNA_inputs[0][0]']            
                                                                                                  
 h0_miRNA (Biological_module)   (None, 238)        

177/177 [==============================] - 0s 270us/sample - loss: 0.5771 - acc: 0.7853
Epoch 55/130
177/177 [==============================] - 0s 231us/sample - loss: 0.5715 - acc: 0.7853
Epoch 56/130
177/177 [==============================] - 0s 220us/sample - loss: 0.5660 - acc: 0.7797
Epoch 57/130
177/177 [==============================] - 0s 220us/sample - loss: 0.5610 - acc: 0.7966
Epoch 58/130
177/177 [==============================] - 0s 242us/sample - loss: 0.5558 - acc: 0.8023
Epoch 59/130
177/177 [==============================] - 0s 254us/sample - loss: 0.5515 - acc: 0.8023
Epoch 60/130
177/177 [==============================] - 0s 282us/sample - loss: 0.5462 - acc: 0.8079
Epoch 61/130
177/177 [==============================] - 0s 259us/sample - loss: 0.5409 - acc: 0.8079
Epoch 62/130
177/177 [==============================] - 0s 265us/sample - loss: 0.5365 - acc: 0.8136
Epoch 63/130
177/177 [==============================] - 0s 248us/sample - loss: 0.5329 - acc: 0.8136
Epo

D:\Anaconda3\lib\site-packages\keras\engine\training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
